In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import sample_rasters as sr

In [2]:
aois = ['campus_lagoon','carpinteria','capitan','gaviota']

n_polygons = []
area_ice = []
n_pixels = []

for aoi in aois:

    fp = os.path.join(os.getcwd(),
                      'FINAL_iceplant_polygons',
                      aoi+'_FINAL_iceplant_polygons',
                      aoi+'_FINAL_iceplant_polygons'+'.shp')

    df = gpd.read_file(fp)

    n_polygons.append(len(df))

    # convert to same crs as raster to properly calculate area of polygons
    itemid = df.aoi[0]
    crs = sr.get_crs_from_itemid(itemid)
    df.to_crs(crs, inplace=True)
    df['area'] = [poly.area for poly in df.geometry] # in m^2
    
    area_ice.append(sum(df.loc[df['iceplant']==1].area))    
    
    # get area of a single pixel from raster resolution  
    item = sr.get_item_from_id(itemid)
    rast_reader = sr.get_raster_from_item(item)
    pixel_size = rast_reader.res[0]*rast_reader.res[1]
    
    pixels_in_polys = df.geometry.apply(lambda p: int((p.area/pixel_size))).to_numpy()
    n_pixels.append(sum(pixels_in_polys))



In [6]:
D = { 
    'aoi': aois, 
    'n_polys' : n_polygons,
    'ice_area_m2' : area_ice,
    'ice_pixels': n_pixels
    }
df_all_polys = pd.DataFrame(D)
df_all_polys

,aoi,n_polys,ice_area_m2,ice_pixels
0,campus_lagoon,102,15114.882469,41932
1,carpinteria,129,16770.546530,46521
2,capitan,36,7783.302426,21603
3,gaviota,28,2043.185018,5662


In [7]:
df_all_polys.ice_pixels.sum()

115718

In [8]:
df_all_polys.n_polys.sum()

295

In [ ]:
sum(df_all_polys.ice_area_m2)/10**6